In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [4]:
data = pd.read_csv("/home/jishnu/Desktop/ Data_Sets/unclean_smartwatch_health_data.csv")